## generate ip level target list for all /24 prefixes

In [7]:
import pickle

from ipaddress import IPv4Network
from random import randint

from default import ADDRESS_FILE, HITLIST_FILE, ANCHORS_FILE

In [3]:
verfploeter_hitlist_file = ADDRESS_FILE

targets_per_prefix = {}

with open(verfploeter_hitlist_file, "r") as f:
    for i, row in enumerate(f.readlines()[1:]):
        row = row.split("\t")

        # get prefix from hex value
        prefix_hex = row[0]
        prefix = ["".join(x) for x in zip(*[iter(prefix_hex)]*2)]
        prefix = [int(x, 16) for x in prefix]
        prefix = ".".join(str(x) for x in prefix)

        target_list = row[-1].strip("\n")
        target_list = target_list.split(",")

        # parse and save targets
        if target_list[0] != '-':
            for i, target in enumerate(target_list):
                target_list[i] = prefix.split(".")[:-1]
                target_list[i].append(str(int(target, 16)))
                target_list[i] = ".".join(target_list[i])

            try:
                targets_per_prefix[prefix].extend(target_list)
            except KeyError:
                targets_per_prefix[prefix] = [target]

In [10]:
print("target hitlist")
for i, prefix in enumerate(targets_per_prefix):
    if i > 10:
        break
    print("prefix:", prefix, "target hitlist:", targets_per_prefix[prefix])

target hitlist
prefix: 1.0.0.0 target hitlist: ['02']
prefix: 1.0.4.0 target hitlist: ['04']
prefix: 1.0.5.0 target hitlist: ['05']
prefix: 1.0.6.0 target hitlist: ['06']
prefix: 1.0.7.0 target hitlist: ['07']
prefix: 1.0.16.0 target hitlist: ['0b']
prefix: 1.0.64.0 target hitlist: ['5f']
prefix: 1.0.65.0 target hitlist: ['f3']
prefix: 1.0.66.0 target hitlist: ['cd']
prefix: 1.0.67.0 target hitlist: ['2b']
prefix: 1.0.68.0 target hitlist: ['83']


In [6]:
out_file = HITLIST_FILE
with open(out_file, "wb") as f:
    pickle.dump(targets_per_prefix, f)

In [8]:
anchor_file = ANCHORS_FILE
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

In [9]:
for i, anchor_ip in enumerate(anchors):
    nb_ip_addresses = 3
    if i > 10:
        break
    prefix_anchor = anchor_ip.split(".")[:-1]
    prefix_anchor.append("0")
    prefix_anchor = ".".join(prefix_anchor)
    ip_list = []
    try:
        ip_list = targets_per_prefix[prefix_anchor]
    except KeyError:
        pass

    if len(ip_list) < nb_ip_addresses:

        ip = anchor_ip.split(".")[:2]
        ip.extend(['0', '0'])
        ip = ".".join(ip)
        prefix_target = IPv4Network(ip + '/' + str(24))

        ip_list.extend([str(prefix_target[randint(1, 254)])
                       for _ in range(0, nb_ip_addresses - len(ip_list))])
    print(f"ip list for anchor {prefix_anchor}: {ip_list}")

ip list for anchor 213.225.160.0: ['9f', '213.225.0.81', '213.225.0.159']
ip list for anchor 145.220.0.0: ['1d', '145.220.0.199', '145.220.0.245']
ip list for anchor 5.28.0.0: ['0e', '5.28.0.4', '5.28.0.189']
ip list for anchor 193.171.255.0: ['09', '193.171.0.196', '193.171.0.104']
ip list for anchor 192.65.184.0: ['45', '192.65.0.191', '192.65.0.248']
ip list for anchor 185.42.136.0: ['03', '185.42.0.90', '185.42.0.58']
ip list for anchor 197.80.104.0: ['23', '197.80.0.208', '197.80.0.114']
ip list for anchor 200.7.84.0: ['04', '200.7.0.211', '200.7.0.36']
ip list for anchor 76.26.115.0: ['c2', '76.26.0.184', '76.26.0.40']
ip list for anchor 194.150.191.0: ['05', '194.150.0.32', '194.150.0.46']
ip list for anchor 199.10.66.0: ['199.10.0.91', '199.10.0.186', '199.10.0.245']
